In [4]:
data_name_list = ['mnist', 'fashion_mnist', 'cifar10', 'cifar100', 'resisc45']
class_list = [10, 10, 10, 100, 45]

k = 0
data_name = data_name_list[k]
num_class = class_list[k]

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

tf.enable_eager_execution()

def normalize(dataset):    
    image, label = tf.cast(dataset['image'], tf.float32) / 255.0, dataset['label']
    return image, label

train_data, test_data = tfds.load(
    data_name, 
    #split=(tfds.Split.TRAIN, tfds.Split.TEST), 
    shuffle_files=False, as_supervised=False)

iterator_train = iter(train_data.map(normalize).batch(50))
iterator_test = iter(test_data.map(normalize))

train = [] # [i][j] i = i번째 배치, j = 0은 이미지, 1은 라벨 
test = []

while True:
    try:
        train.append(next(iterator_train))
    except StopIteration:
        break
while True:
    try:
        test.append(next(iterator_test))
    except StopIteration:
        break
        

In [27]:
train_image = [] # [i][j] i= i번째 배치, j= j번째 이미지
train_label = []

test_image = [] # [i][j] i= i번째 배치, j= j번째 이미지
test_label = []

#train
for i in range(len(train)):
    
    target_train = train[i][1].numpy()
    
    encoding = np.eye(num_class)[target_train]
    train_image.append(train[i][0].numpy().reshape((-1,32*32*3)))
    train_label.append(encoding)

#test
for i in range(len(test)):    
    target_test = test[i][1].numpy()
    
    encoding = np.eye(num_class)[target_test]
    test_image.append(test[i][0].numpy().reshape((-1,32*32*3)))
    test_label.append(encoding)
    
test_image = np.array(test_image).reshape(len(test_image), test_image[0].shape[1])
test_label = np.array(test_label)

In [28]:
len(train_image), train_image[0].shape

(1000, (50, 3072))

In [29]:
len(test_image), test_image[0].shape

(10000, (3072,))

In [30]:
np.array(test_label).shape

(10000, 100)

In [31]:
np.save('C:\\Users\\Yang\\Desktop\\admm-pruning-master\\data\\'+data_name+'_train_image', train_image)
np.save('C:\\Users\\Yang\\Desktop\\admm-pruning-master\\data\\'+data_name+'_train_label', train_label)
np.save('C:\\Users\\Yang\\Desktop\\admm-pruning-master\\data\\'+data_name+'_test_image', test_image)
np.save('C:\\Users\\Yang\\Desktop\\admm-pruning-master\\data\\'+data_name+'_test_label', test_label)

In [4]:
n_image = np.load('C:\\Users\\Yang\\Desktop\\admm-pruning-master\\image_'+data_name+'.npy')
n_label = np.load('C:\\Users\\Yang\\Desktop\\admm-pruning-master\\label_'+data_name+'.npy')

In [ ]:
len(n_image),len(n_image[0]), len(n_image[0][1]), len(n_image[0][0][0]), len(n_image[0][0][0][0])

########################################

In [44]:
# Copyright 2015 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

"""A deep MNIST classifier using convolutional layers.

See extensive documentation at
https://www.tensorflow.org/get_started/mnist/pros
"""
# Disable linter warnings to maintain consistency with tutorial.
# pylint: disable=invalid-name
# pylint: disable=g-bad-import-order

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys
import tempfile
from model import create_model
from solver import create_admm_solver
from tensorflow.examples.tutorials.mnist import input_data
from prune_utility import apply_prune_on_grads,apply_prune,get_configuration,projection
import tensorflow as tf
import numpy as np
from numpy import linalg as LA
import tensorflow as tf
#import tensorflow_datasets as tfds
import numpy as np
import random

#tf.disable_eager_execution()

FLAGS = None
# pruning ratio

data_name_list = ['mnist', 'fashion_mnist', 'Cifar10', 'Cifar100']

data_name = data_name_list[0]
num_class = 10

prune_configuration = get_configuration()
dense_w = {}
P1 = prune_configuration.P1
P2 = prune_configuration.P2
P3 = prune_configuration.P3
P4 = prune_configuration.P4

prune_configuration.display()


def main(_):
  # Import data
  train_image = np.load('C:\\Users\\Yang\\Desktop\\admm-pruning-master\\data\\'+data_name+'_train_image'+'.npy')
  train_label = np.load('C:\\Users\\Yang\\Desktop\\admm-pruning-master\\data\\'+data_name+'_train_label'+'.npy')
  test_image = np.load('C:\\Users\\Yang\\Desktop\\admm-pruning-master\\data\\'+data_name+'_test_image'+'.npy')
  test_label = np.load('C:\\Users\\Yang\\Desktop\\admm-pruning-master\\data\\'+data_name+'_test_label'+'.npy')

  model = create_model()
  x = model.x
  y_ = model.y_
  cross_entropy = model.cross_entropy
  layers = model.layers
  logits = model.logits
  solver = create_admm_solver(model)
  keep_prob = model.keep_prob
  train_step = solver.train_step
  train_step1 = solver.train_step1
  
  W_conv1 = model.W_conv1
  W_conv2 = model.W_conv2
  W_fc1 = model.W_fc1
  W_fc2 = model.W_fc2
  
  A = solver.A
  B = solver.B
  C = solver.C
  D = solver.D
  E = solver.E
  F = solver.F
  G = solver.G
  H = solver.H
    

  # 첫 번째 stop condition
  stop_W_Z_Conv1 = [] 
  stop_W_Z_Conv2 = [] 
  stop_W_Z_FC1 = []
  stop_W_Z_FC2 = []
  stop_W_Z = [] #전체 값  
    
  # 두 번째 stop condition
  stop_Z_Z_Conv1 = [] 
  stop_Z_Z_Conv2 = [] 
  stop_Z_Z_FC1 = [] 
  stop_Z_Z_FC2 = [] 
  stop_W_Z = [] #전체 값  
    
    
  my_trainer = tf.train.AdamOptimizer(1e-3)
  grads = my_trainer.compute_gradients(cross_entropy)
    
  with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y_, 1))
    correct_prediction = tf.cast(correct_prediction, tf.float32)
  accuracy = tf.reduce_mean(correct_prediction)

  graph_location = tempfile.mkdtemp()
  print('Saving graph to: %s' % graph_location)
  train_writer = tf.summary.FileWriter(graph_location)
  train_writer.add_graph(tf.get_default_graph())

  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(1000):
        j = random.randrange(0, len(train_image), 1)
        batch = []
        batch.append(train_image[j])
        batch.append(train_label[j])
        if i % 100 == 0:
            train_accuracy = accuracy.eval(feed_dict={
                x: batch[0], y_: batch[1], keep_prob: 1.0})
            print('step %d, training accuracy %g' % (i, train_accuracy))
        train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

    print('test accuracy %g' % accuracy.eval(feed_dict={
        x: test_image, y_: test_label, keep_prob: 1.0}))
    
    pretrained_weight = np.array([sess.run(W_conv1), sess.run(W_conv2), sess.run(W_fc1), sess.run(W_fc2)])
    
    # pretrain된 weight 저장 
    np.save('C:\\Users\\Yang\\Desktop\\admm-pruning-master\\weight\\'+'pretrained_'+data_name, pretrained_weight)
    
#     # pretrain 모델 저장
#     saver = tf.train.Saver()
#     saver.save(sess, 'C:\\Users\\Yang\\Desktop\\admm-pruning-master\\model\\'+data_name+'_pretrained')
    
    Z1 = sess.run(W_conv1)
    Z1 = projection(Z1, percent=P1)

    U1 = np.zeros_like(Z1)

    Z2 = sess.run(W_conv2)
    Z2 = projection(Z2, percent=P2)

    U2 = np.zeros_like(Z2)

    Z3 = sess.run(W_fc1)
    Z3 = projection(Z3, percent=P3)

    U3 = np.zeros_like(Z3)

    Z4 = sess.run(W_fc2)
    Z4 = projection(Z4, percent=P4)

    U4 = np.zeros_like(Z4)
    
    for j in range(20):
        for i in range(500):
            j = random.randrange(0, len(train_image), 1)
            batch = []
            batch.append(train_image[j])
            batch.append(train_label[j])
            
            if i % 100 == 0:
                train_accuracy = accuracy.eval(feed_dict={
                    x: batch[0], y_: batch[1], keep_prob: 1.0})
                print('step %d, training accuracy %g' % (i, train_accuracy))
            train_step1.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 1.0, A:Z1, B:U1, C:Z2, D:U2, E:Z3, F:U3, G:Z4, H:U4})
        
        #Z k번째값 저장
        Z_k = [Z1, Z2, Z3, Z4]
        
        Z1 = sess.run(W_conv1) + U1
        Z1 = projection(Z1, percent=P1)

        U1 = U1 + sess.run(W_conv1) - Z1

        Z2 = sess.run(W_conv2) + U2
        Z2 = projection(Z2, percent=P2)

        U2 = U2 + sess.run(W_conv2) - Z2

        Z3 = sess.run(W_fc1) + U3
        Z3 = projection(Z3, percent=P3)

        U3 = U3 + sess.run(W_fc1) - Z3

        Z4 = sess.run(W_fc2) + U4
        Z4 = projection(Z4, percent=P4)

        U4 = U4 + sess.run(W_fc2) - Z4
        
        
        print('test accuracy %g' % accuracy.eval(feed_dict={
            x: test_image, y_: test_label, keep_prob: 1.0}))
        
        #stop condition 값 저장 
        stop_W_Z_Conv1.append(LA.norm(sess.run(W_conv1) - Z1))
        stop_W_Z_Conv2.append(LA.norm(sess.run(W_conv2) - Z2))
        stop_W_Z_FC1.append(LA.norm(sess.run(W_fc1) - Z3))
        stop_W_Z_FC2.append(LA.norm(sess.run(W_fc2) - Z4))
        
        
        stop_Z_Z_Conv1.append(LA.norm(Z_k[0]-Z1))
        stop_Z_Z_Conv2.append(LA.norm(Z_k[1]-Z2))
        stop_Z_Z_FC1.append(LA.norm(Z_k[2]-Z3))
        stop_Z_Z_FC2.append(LA.norm(Z_k[3]-Z4))
        
#         print(LA.norm(sess.run(W_conv1) - Z1))
#         print(LA.norm(sess.run(W_conv2) - Z2))
#         print(LA.norm(sess.run(W_fc1) - Z3))
#         print(LA.norm(sess.run(W_fc2) - Z4))

    
    
    stop_W_Z = [stop_W_Z_Conv1, stop_W_Z_Conv2, stop_W_Z_FC1, stop_W_Z_FC2]
    stop_Z_Z = [stop_Z_Z_Conv1, stop_Z_Z_Conv2, stop_Z_Z_FC1, stop_Z_Z_FC2]
    
    np.save('C:\\Users\\Yang\\Desktop\\admm-pruning-master\\convergence\\'+data_name+'_stopWZ', stop_W_Z)
    np.save('C:\\Users\\Yang\\Desktop\\admm-pruning-master\\convergence\\'+data_name+'_stopZZ', stop_Z_Z)
    
    
    dense_w['conv1/W_conv1'] = W_conv1
    dense_w['conv2/W_conv2'] = W_conv2
    dense_w['fc1/W_fc1'] = W_fc1
    dense_w['fc2/W_fc2'] = W_fc2
    
    dict_nzidx = apply_prune(dense_w,sess)
    admm_weight = np.array([sess.run(W_conv1), sess.run(W_conv2), sess.run(W_fc1), sess.run(W_fc2)])
    np.save('C:\\Users\\Yang\\Desktop\\admm-pruning-master\\weight\\'+'admm_'+data_name, admm_weight)
    
    
    print ("checking space dictionary")
    print (dict_nzidx.keys())
    grads = apply_prune_on_grads(grads,dict_nzidx)
    apply_gradient_op = my_trainer.apply_gradients(grads)
    for var in tf.global_variables():
                if tf.is_variable_initialized(var).eval() == False:
                    sess.run(tf.variables_initializer([var]))
    print ("start retraining after pruning")
    for i in range(2000):
        j = random.randrange(0, len(train_image), 1)
        batch = []
        batch.append(train_image[j])
        batch.append(train_label[j])
        if i % 100 == 0:
            train_accuracy = accuracy.eval(feed_dict={
                x: batch[0], y_: batch[1], keep_prob: 1.0})
            print('step %d, training accuracy %g' % (i, train_accuracy))

        apply_gradient_op.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
    
    retrain_weight = np.array([sess.run(W_conv1), sess.run(W_conv2), sess.run(W_fc1), sess.run(W_fc2)])
    np.save('C:\\Users\\Yang\\Desktop\\admm-pruning-master\\weight\\'+'retrained_'+data_name, retrain_weight)
    
    print('test accuracy %g' % accuracy.eval(feed_dict={
          x: test_image, y_: test_label, keep_prob: 1.0}))
    print(np.sum(sess.run(W_conv1)!=0))
    print(np.sum(sess.run(W_conv2) != 0))
    print(np.sum(sess.run(W_fc1) != 0))
    print(np.sum(sess.run(W_fc2) != 0))
    # do the saving.
    saver = tf.train.Saver()
    saver.save(sess,"./lenet_5_pruned_model.ckpt")
if __name__ == '__main__':
  parser = argparse.ArgumentParser()
  parser.add_argument('--data_dir', type=str,
                      default='/tmp/tensorflow/mnist/input_data',
                      help='Directory for storing input data')
  FLAGS, unparsed = parser.parse_known_args()
  tf.app.run(main=main, argv=[sys.argv[0]] + unparsed)
  


P1 is 80.000000
P2 is 92.000000
P3 is 99.100000
P4 is 93.000000
Saving graph to: C:\Users\Yang\AppData\Local\Temp\tmp7__dao3o
step 0, training accuracy 0.12
step 100, training accuracy 0.94
step 200, training accuracy 0.98
step 300, training accuracy 0.86
step 400, training accuracy 0.96
step 500, training accuracy 0.96
step 600, training accuracy 0.98
step 700, training accuracy 0.98
step 800, training accuracy 0.98
step 900, training accuracy 1
test accuracy 0.9845
percentile 0.13910980522632602
percentile 0.15807356655597687
percentile 0.19487644244730482
percentile 0.16286328852176668
step 0, training accuracy 0.98
step 100, training accuracy 1
step 200, training accuracy 0.94
step 300, training accuracy 0.98
step 400, training accuracy 0.98
percentile 0.14124430716037753
percentile 0.15557461798191072
percentile 0.17708150936663178
percentile 0.16698487162590028
test accuracy 0.9854
step 0, training accuracy 0.98
step 100, training accuracy 1
step 200, training accuracy 0.98
step 

ValueError: Tried to convert 'y' to a tensor and failed. Error: None values not supported.

In [5]:
# Copyright 2015 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

"""A deep MNIST classifier using convolutional layers.

See extensive documentation at
https://www.tensorflow.org/get_started/mnist/pros
"""
# Disable linter warnings to maintain consistency with tutorial.
# pylint: disable=invalid-name
# pylint: disable=g-bad-import-order

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys
import tempfile
from model import create_model
from solver import create_admm_solver
from tensorflow.examples.tutorials.mnist import input_data
from prune_utility import apply_prune_on_grads,apply_prune,get_configuration,projection
import tensorflow as tf
import numpy as np
from numpy import linalg as LA

FLAGS = None
# pruning ratio


prune_configuration = get_configuration()
dense_w = {}
P1 = prune_configuration.P1
P2 = prune_configuration.P2
P3 = prune_configuration.P3
P4 = prune_configuration.P4

prune_configuration.display()


def main(_):
  # Import data
  mnist = input_data.read_data_sets(FLAGS.data_dir, one_hot=True)

  model = create_model()
  x = model.x
  y_ = model.y_
  cross_entropy = model.cross_entropy
  layers = model.layers
  logits = model.logits
  solver = create_admm_solver(model)
  keep_prob = model.keep_prob
  train_step = solver.train_step
  train_step1 = solver.train_step1
  
  W_conv1 = model.W_conv1
  W_conv2 = model.W_conv2
  W_fc1 = model.W_fc1
  W_fc2 = model.W_fc2
  
  A = solver.A
  B = solver.B
  C = solver.C
  D = solver.D
  E = solver.E
  F = solver.F
  G = solver.G
  H = solver.H

  my_trainer = tf.train.AdamOptimizer(1e-3)
  grads = my_trainer.compute_gradients(cross_entropy)
    
  with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y_, 1))
    correct_prediction = tf.cast(correct_prediction, tf.float32)
  accuracy = tf.reduce_mean(correct_prediction)

  graph_location = tempfile.mkdtemp()
  print('Saving graph to: %s' % graph_location)
  train_writer = tf.summary.FileWriter(graph_location)
  train_writer.add_graph(tf.get_default_graph())

  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(500):
      batch = mnist.train.next_batch(50)
      if i % 100 == 0:
        train_accuracy = accuracy.eval(feed_dict={
            x: batch[0], y_: batch[1], keep_prob: 1.0})
        print('step %d, training accuracy %g' % (i, train_accuracy))
      train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

    print('test accuracy %g' % accuracy.eval(feed_dict={
        x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

    Z1 = sess.run(W_conv1)
    Z1 = projection(Z1, percent=P1)

    U1 = np.zeros_like(Z1)

    Z2 = sess.run(W_conv2)
    Z2 = projection(Z2, percent=P2)

    U2 = np.zeros_like(Z2)

    Z3 = sess.run(W_fc1)
    Z3 = projection(Z3, percent=P3)

    U3 = np.zeros_like(Z3)

    Z4 = sess.run(W_fc2)
    Z4 = projection(Z4, percent=P4)

    U4 = np.zeros_like(Z4)

    for j in range(10):
        for i in range(1000):
            batch = mnist.train.next_batch(50)
            if i % 100 == 0:
                train_accuracy = accuracy.eval(feed_dict={
                    x: batch[0], y_: batch[1], keep_prob: 1.0})
                print('step %d, training accuracy %g' % (i, train_accuracy))
            train_step1.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 1.0, A:Z1, B:U1, C:Z2, D:U2, E:Z3, F:U3, G:Z4, H:U4})
        Z1 = sess.run(W_conv1) + U1
        Z1 = projection(Z1, percent=P1)

        U1 = U1 + sess.run(W_conv1) - Z1

        Z2 = sess.run(W_conv2) + U2
        Z2 = projection(Z2, percent=P2)

        U2 = U2 + sess.run(W_conv2) - Z2

        Z3 = sess.run(W_fc1) + U3
        Z3 = projection(Z3, percent=P3)

        U3 = U3 + sess.run(W_fc1) - Z3

        Z4 = sess.run(W_fc2) + U4
        Z4 = projection(Z4, percent=P4)

        U4 = U4 + sess.run(W_fc2) - Z4

        print('test accuracy %g' % accuracy.eval(feed_dict={
            x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))
        print(LA.norm(sess.run(W_conv1) - Z1))
        print(LA.norm(sess.run(W_conv2) - Z2))
        print(LA.norm(sess.run(W_fc1) - Z3))
        print(LA.norm(sess.run(W_fc2) - Z4))

    dense_w['conv1/W_conv1'] = W_conv1
    dense_w['conv2/W_conv2'] = W_conv2
    dense_w['fc1/W_fc1'] = W_fc1
    dense_w['fc2/W_fc2'] = W_fc2
    
    dict_nzidx = apply_prune(dense_w,sess)
    print ("checking space dictionary")
    print (dict_nzidx.keys())
    grads = apply_prune_on_grads(grads,dict_nzidx)
    apply_gradient_op = my_trainer.apply_gradients(grads)
    for var in tf.global_variables():
                if tf.is_variable_initialized(var).eval() == False:
                    sess.run(tf.variables_initializer([var]))
    print ("start retraining after pruning")
    for i in range(2000):
      batch = mnist.train.next_batch(50)
      if i % 100 == 0:
        train_accuracy = accuracy.eval(feed_dict={
            x: batch[0], y_: batch[1], keep_prob: 1.0})
        print('step %d, training accuracy %g' % (i, train_accuracy))

      apply_gradient_op.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
    print('test accuracy %g' % accuracy.eval(feed_dict={
          x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))
    print(np.sum(sess.run(W_conv1)!=0))
    print(np.sum(sess.run(W_conv2) != 0))
    print(np.sum(sess.run(W_fc1) != 0))
    print(np.sum(sess.run(W_fc2) != 0))
    # do the saving.
    saver = tf.train.Saver()
    saver.save(sess,"./lenet_5_pruned_model.ckpt")
if __name__ == '__main__':
  parser = argparse.ArgumentParser()
  parser.add_argument('--data_dir', type=str,
                      default='/tmp/tensorflow/mnist/input_data',
                      help='Directory for storing input data')
  FLAGS, unparsed = parser.parse_known_args()
  tf.app.run(main=main, argv=[sys.argv[0]] + unparsed)
  


P1 is 80.000000
P2 is 92.000000
P3 is 99.100000
P4 is 93.000000
Extracting /tmp/tensorflow/mnist/input_data\train-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data\train-labels-idx1-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data\t10k-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data\t10k-labels-idx1-ubyte.gz


RuntimeError: tf.placeholder() is not compatible with eager execution.

In [4]:
tf.compat.v1.disable_eager_execution()